In [39]:
import requests
import pandas as pd

In [40]:
#KEYS API_KEY can be generated from developers.google.com and channel id is the id of the the youtube account
API_KEY = ''
CHANNEL_ID = 'UC8butISFwT-Wl7EV0hUK0BQ'

In [41]:
def get_video_details(video_id):
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?part=statistics&id="+video_id+"&key="+API_KEY
    response_video_stats = requests.get(url_video_stats).json()
    
    video_view_count = response_video_stats['items'][0]['statistics']['viewCount']
    video_like_count = response_video_stats['items'][0]['statistics']['likeCount']
    video_comment_count = response_video_stats['items'][0]['statistics']['commentCount']
    
    return video_view_count,video_like_count,video_comment_count
    

In [42]:
def get_video(df):
    #MAKE API CALL TO FETCH ALL VIDEOS
    pageToken = ""
    url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000"

    response = requests.get(url).json()
    for video in response['items']:
        if video['id']['kind'] == 'youtube#video':
            video_id = video['id']['videoId']
            video_title = video['snippet']['title']
            upload_date = video['snippet']['publishedAt']
            upload_date = str(upload_date).split("T")[0]

            video_view_count,video_like_count,video_comment_count = get_video_details(video_id)

            df = df.append({
                'video_id': video_id,
                'video_title': video_title,
                'upload_date': upload_date,
                'view_count': video_view_count,
                'like_count': video_like_count,
                'comment_count': video_comment_count}, ignore_index = True)
            
    return df

In [120]:
#MAIN

df2 = pd.DataFrame(columns=['video_id','video_title','upload_date','view_count','like_count','comment_count'])

df2 = get_video(df2)
df2.head()

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,8124kv-632k,Python Tutorial for Beginners,2022-03-15,48987,2612,108
1,AplluksKvzI,CNCF Kubernetes and Cloud Native Associate Cer...,2022-03-10,64511,2365,125
2,RZ4p-saaQkc,Vim Tutorial for Beginners,2022-03-09,89075,3339,178
3,kUMe1FH4CHE,Learn HTML – Full Tutorial for Beginners (2022),2022-03-08,86109,3684,123
4,hN4Q04ODOEw,Build a Live Stream Flutter App – Android &amp...,2022-03-07,54119,1956,69


In [47]:
pip install PyMySQL

In [57]:
import pymysql.cursors

In [121]:
def conect_to_db(host_name,db_name,user_name,password,port):
    try:
        connection = pymysql.connect(user=user_name,password=password,host=host_name,database=db_name,port=port,cursorclass=pymysql.cursors.DictCursor)
    except pymysql.OperationalError as e:
        raise e
    else:
        print('connected to db!')
    return connection
            

In [122]:
#connecting to db

host_name = ''
db_name = ''
user_name = ''
password = ''
port=

conn = conect_to_db(host_name,db_name,user_name,password,port)

connected to db!


In [169]:
#create table
def create_table(cursor):
    create_table_query = ("""CREATE TABLE IF NOT EXISTS videos (video_id  VARCHAR(255) PRIMARY KEY,
                                                                video_title TEXT NOT NULL,
                                                                upload_date DATE NOT NULL,
                                                                view_count INTEGER NOT NULL,
                                                                like_count INTEGER NOT NULL,
                                                                comment_count INTEGER NOT NULL)""")
    cursor.execute(create_table_query)
    
#checks for any new videos
def check_if_video_exists(cursor,video_id):
    query = ("""SELECT video_id FROM videos WHERE video_id = %s""")
    cursor.execute(query,(video_id))
    
    return cursor.fetchone() is not None

#helper function to update the values of existing data
def update_row(cursor,video_id,video_title,view_count,like_count,comment_count):
    query = ("""UPDATE videos 
            SET video_title = %s,
                view_count = %s,
                like_count = %s,
                comment_count = %s
            WHERE video_id = %s;""")
    vars_to_update = (video_title, view_count, like_count, comment_count, video_id)
    cursor.execute(query, vars_to_update)
    
#main func to update the existing values and check for new vids
def update_db(cursor,df):
    tmp_df = pd.DataFrame(columns=['video_id','video_title','upload_date','view_count','like_count','comment_count'])
    for i,row in df.iterrows():
        if check_if_video_exists(cursor,row['video_id']): #if video already exists this will update the values
            update_row(cursor,row['video_id'],row['video_title'],row['view_count'],row['like_count'],row['comment_count'])
        else: #if the video does not exists in db it will appeneded through a df
            tmp_df = tmp_df.append(row)
    return tmp_df

#helper func to insert new data into the table
def insert_into_table(cursor,video_id,video_title,upload_date,view_count,like_count,comment_count):
    insert_into_videos = ("""INSERT INTO videos(video_id,video_title,upload_date,view_count,like_count,comment_count)
                             VALUES(%s,%s,%s,%s,%s,%s);""")
    insert_vars = (video_id,video_title,upload_date,view_count,like_count,comment_count)
    cursor.execute(insert_into_videos, insert_vars)

#helper func used in inserting new data
def append_from_df_to_db(cursor,df):    
    for i,row in df.iterrows():
        insert_into_table(cursor,row['video_id'],row['video_title'],row['upload_date'],row['view_count'],row['like_count'],row['comment_count'])

In [170]:
cursor = conn.cursor()

In [171]:
create_table(cursor)

In [172]:
new_vid_df = update_db(cursor,df2)

In [173]:
append_from_df_to_db(cursor,new_vid_df)

In [174]:
conn.commit()